In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_csv('data/gsearch_jobs.csv')

In [3]:
df_raw.head()
df_raw.info()
df_raw.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42151 entries, 0 to 42150
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           42151 non-null  int64  
 1   index                42151 non-null  int64  
 2   title                42151 non-null  object 
 3   company_name         42151 non-null  object 
 4   location             42122 non-null  object 
 5   via                  42142 non-null  object 
 6   description          42151 non-null  object 
 7   extensions           42151 non-null  object 
 8   job_id               42151 non-null  object 
 9   thumbnail            25015 non-null  object 
 10  posted_at            42151 non-null  object 
 11  schedule_type        41974 non-null  object 
 12  work_from_home       19077 non-null  object 
 13  salary               7220 non-null   object 
 14  search_term          42151 non-null  object 
 15  date_time            42151 non-null 

(42151, 27)

In [4]:
#cleaning section
df = df_raw.drop(
    ["job_id",
      "thumbnail" ,
      "commute_time",
      "salary_pay", # as we have min-max
      "salary_hourly",
      "Unnamed: 0",
      "posted_at",
       ],
      axis = 1
      )
df.tail()

,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary,search_term,date_time,search_location,salary_rate,salary_avg,salary_min,salary_max,salary_yearly,salary_standardized,description_tokens
42146,524,Marketing Data & BI Analyst II,EDWARD JONES,"Houstonia, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '76,798–130,764 a year', 'Ful...",Full-time,NaN,"76,798–130,764 a year",data analyst,2022-11-04 03:40:23.706734,United States,a year,103781.0,76798.0,130764.0,103781.0,103781.0,"['snowflake', 'sql', 'excel', 'tableau', 'powe..."
42147,525,Lead-Data Analyst,EDWARD JONES,"Marshfield, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,NaN,"106,916–182,047 a year",data analyst,2022-11-24 04:00:08.710801,United States,a year,144481.5,106916.0,182047.0,144481.5,144481.5,[]
42148,526,Lead-Data Analyst,EDWARD JONES,"High Point, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,NaN,"106,916–182,047 a year",data analyst,2022-12-07 04:00:12.563831,United States,a year,144481.5,106916.0,182047.0,144481.5,144481.5,[]
42149,527,Lead-Data Analyst,EDWARD JONES,"Calhoun, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,NaN,"106,916–182,047 a year",data analyst,2022-12-08 04:00:15.975728,United States,a year,144481.5,106916.0,182047.0,144481.5,144481.5,[]
42150,528,Institutional Credit Management - Lending Data...,Citi,United States,via My ArkLaMiss Jobs,The Institutional Credit Management (ICM) grou...,"['24 hours ago', '105,850–158,780 a year', 'Fu...",Full-time,NaN,"105,850–158,780 a year",data analyst,2023-01-25 04:00:26.521124,United States,a year,132315.0,105850.0,158780.0,132315.0,132315.0,"['cognos', 'tableau']"


In [5]:
# converting to types, "description_tokens" still object
df[["title", "description", "company_name", "location", "via", "extensions", "schedule_type", "search_term", "search_location", "salary", "salary_rate"]] = df[["title", "description", "company_name", "location", "via", "extensions", "schedule_type", "search_term", "search_location", "salary", "salary_rate"]].astype("string")
df["work_from_home"] = df["work_from_home"].astype("bool")
df["date_time"] = pd.to_datetime(df["date_time"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42151 entries, 0 to 42150
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                42151 non-null  int64         
 1   title                42151 non-null  string        
 2   company_name         42151 non-null  string        
 3   location             42122 non-null  string        
 4   via                  42142 non-null  string        
 5   description          42151 non-null  string        
 6   extensions           42151 non-null  string        
 7   schedule_type        41974 non-null  string        
 8   work_from_home       42151 non-null  bool          
 9   salary               7220 non-null   string        
 10  search_term          42151 non-null  string        
 11  date_time            42151 non-null  datetime64[ns]
 12  search_location      42151 non-null  string        
 13  salary_rate          7220 non-n

In [6]:
#covert to lower strings
# title and description changed back to object type --> "Data Analyst II" is now "data analyst ii"
df["title"] = df["title"].apply(lambda x: x.lower() if isinstance(x, str) else x)
df["description"] = df["description"].apply(lambda x: x.lower() if isinstance(x, str) else x)

# SKILLNER package cant  handle back ticks (’), replacing in df.description
df["description_cleaned"] = df["description"].replace('’', " ")

df["via"] = df["via"].apply(lambda x: x.lower() if isinstance(x, str) else x)

df[["title", "description","description_cleaned", "via"]] = df[["title", "description","description_cleaned", "via"]].astype("string")
df.head()
#df.info()

,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary,...,date_time,search_location,salary_rate,salary_avg,salary_min,salary_max,salary_yearly,salary_standardized,description_tokens,description_cleaned
0,0,data analyst,Cisco Meraki,Anywhere,via linkedin,"as the leader in cloud-managed it, cisco merak...","['22 hours ago', 'Work from home', 'Full-time'...",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,"['sql', 'aws', 'tableau']","as the leader in cloud-managed it, cisco merak..."
1,1,entry level - business data analyst (remote),Midwest Staffing,Anywhere,via linkedin,as a senior business analyst you will contribu...,"['53 minutes ago', 'Work from home', 'Full-tim...",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,[],as a senior business analyst you will contribu...
2,2,data analyst/researcher,"Amyx, Inc.",United States,via indeed,overview: amyx is seeking to hire a data anal...,"['3 hours ago', 'Full-time']",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,"['azure', 'sql', 'excel']",overview: amyx is seeking to hire a data anal...
3,3,data analyst/ai expert to help build a website...,Upwork,Anywhere,via upwork,i am looking for someone to help me build an a...,"['23 hours ago', 'Work from home', 'Contractor']",Contractor,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,['excel'],i am looking for someone to help me build an a...
4,4,data analyst,Bank Street College of Education,United States,via higher education recruitment consortium (h...,position vacancy – data analyst to support the...,"['20 hours ago', 'Full-time']",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,"['excel', 'powerpoint']",position vacancy – data analyst to support the...


In [7]:
#Getting to know the dataset
display(df["location"].value_counts())
germany = df["location"].value_counts().get('Germany')
print(germany)
display(df["title"].value_counts())

location
 Anywhere                 11983
  United States            7964
Anywhere                   7094
United States              3660
  Oklahoma City, OK         818
                          ...  
Mountain View, AR             1
Chickasha, OK                 1
  Ada, OK                     1
  Silver Lake, KS             1
Sand Springs, OK              1
Name: count, Length: 712, dtype: Int64

None


title
data analyst                                           5198
senior data analyst                                    1381
data scientist                                          579
data analyst ii                                         530
business data analyst                                   408
                                                       ... 
business intelligence analyst -remote                     1
principal analytical data engineer - healthcare           1
data quality analyst - ( german/ french) - remote         1
business data & reporting analyst – hybrid | 975913       1
lead fp&a analyst- remote, us                             1
Name: count, Length: 15347, dtype: Int64

In [8]:
# checking how many data analyst and scientists
# manually adjusting the filter 
df["is_analyst"] = df["title"].str.contains("analyst", case=False) & ~ (df["title"].str.contains("business analyst", case=False) | df["title"].str.contains("bi analyst", case=False)) # analyst but not business analyst
print(df["is_analyst"].sum())
df["is_bi_analyst"] = df["title"].str.contains("business data analyst", case=False)
print(df["is_bi_analyst"].sum())
df["is_scien"] = df["title"].str.contains("scient" or "scienc", case=False)
print(df["is_scien"].sum())
df.tail()

32022
926
1784


,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary,...,salary_avg,salary_min,salary_max,salary_yearly,salary_standardized,description_tokens,description_cleaned,is_analyst,is_bi_analyst,is_scien
42146,524,marketing data & bi analyst ii,EDWARD JONES,"Houstonia, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '76,798–130,764 a year', 'Ful...",Full-time,True,"76,798–130,764 a year",...,103781.0,76798.0,130764.0,103781.0,103781.0,"['snowflake', 'sql', 'excel', 'tableau', 'powe...","at edward jones, we help clients achieve their...",False,False,False
42147,525,lead-data analyst,EDWARD JONES,"Marshfield, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,True,"106,916–182,047 a year",...,144481.5,106916.0,182047.0,144481.5,144481.5,[],"at edward jones, we help clients achieve their...",True,False,False
42148,526,lead-data analyst,EDWARD JONES,"High Point, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,True,"106,916–182,047 a year",...,144481.5,106916.0,182047.0,144481.5,144481.5,[],"at edward jones, we help clients achieve their...",True,False,False
42149,527,lead-data analyst,EDWARD JONES,"Calhoun, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,True,"106,916–182,047 a year",...,144481.5,106916.0,182047.0,144481.5,144481.5,[],"at edward jones, we help clients achieve their...",True,False,False
42150,528,institutional credit management - lending data...,Citi,United States,via my arklamiss jobs,the institutional credit management (icm) grou...,"['24 hours ago', '105,850–158,780 a year', 'Fu...",Full-time,True,"105,850–158,780 a year",...,132315.0,105850.0,158780.0,132315.0,132315.0,"['cognos', 'tableau']",the institutional credit management (icm) grou...,True,False,False


In [9]:
# testing the filters
# filtering for df["is_analyst"] based on boolean indexing when df[df["is_analyst"]] is true
filtered_df = df[df["is_analyst"]]

# manual inspection
display(filtered_df[["title", "is_analyst"]])

,title,is_analyst
0,data analyst,True
1,entry level - business data analyst (remote),True
2,data analyst/researcher,True
3,data analyst/ai expert to help build a website...,True
4,data analyst,True
...,...,...
42145,sr data analyst,True
42147,lead-data analyst,True
42148,lead-data analyst,True
42149,lead-data analyst,True


In [10]:
#where jobs are from, glassdoor = 0, 
#FIX ME , there was the word indeed, but doesnt show here
df["via"].str.contains("indeed").sum()

df["via"].str.contains("glassdoor").sum()
indexes_with_glassdoor = df[df["via"].str.contains("glassdoor")].index
print(indexes_with_glassdoor)
df["via"].loc[12579]
df["date_time"].loc[12444]

Index([12444, 12579], dtype='int64')


Timestamp('2022-11-04 15:36:57.785729')

In [11]:
# FINAL FILTER, create df_cleaned, finally create df_main
# Decision to integrate every title that is analyst but not business analyst or bi analyst which still can overlap with some other titles that are also in the same title description
df_cleaned = df[(df["is_analyst"] == True)]

print(df_cleaned.shape)

df_main = df_cleaned.copy()

(32022, 24)


In [12]:
#starting tokenization
#a random list of skills to check if tokenization works and for first impressions of the df
skills_list = ["python", "sql", "tableau", "bi tool", "power bi", "aws", "azure", "excel", "powerpoint"]# R als entity!!!  # start with these two skills
#r'\b(r)\b'   --> how to extract the stand-alone r

# create new columns for each skill
for skill in skills_list:
    df_main[skill] = df_main['description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

In [14]:
df_main[["python", "sql", "tableau", "aws", "azure", "excel", "powerpoint", "power bi"]].sum(axis=0).sort_values(ascending = False)

excel         18793
sql           18351
tableau       10113
python         9553
power bi       6616
aws            4198
powerpoint     2916
azure          1779
dtype: int64

In [15]:
df_main.to_csv("data/gsearch_cleaned.csv")